In [1]:
!git clone https://github.com/routsourav1729/CUB_200_2011.git


Cloning into 'CUB_200_2011'...
remote: Enumerating objects: 12195, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 12195 (delta 0), reused 4 (delta 0), pack-reused 12191
Receiving objects: 100% (12195/12195), 1.05 GiB | 34.33 MiB/s, done.
Updating files: 100% (11790/11790), done.


In [2]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=86ff11281a943df4d80728d03a15f6edd97e493328e9e8b56a29c266636705b3
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [ ]:
pip install "torchvision>0.16.0"

In [5]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch import optim

from sklearn.metrics import accuracy_score

In [6]:
# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=299, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=299),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=299),
        transforms.CenterCrop(size=299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [7]:
# Datasets from folders
batch_size = 32
data = {
    'train':
    datasets.ImageFolder(root='/content/CUB_200_2011/train', transform=image_transforms['train']),
    'test':
    datasets.ImageFolder(root='/content/CUB_200_2011/test', transform=image_transforms['test']),
}

# Dataloader iterators, make sure to shuffle
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
}

In [18]:
import torch
import torch.nn as nn
from torchvision import models

# Initialize the EfficientNet-B2 model with pre-trained weights
model = models.efficientnet_b2(weights='IMAGENET1K_V1')

# Determine the number of layers in the model
num_layers = len(list(model.children()))

# Specify the number of layers to freeze (initial convolutional layers)
freeze_layers = 205

# Freeze the initial convolutional layers
for i, (name, param) in enumerate(model.named_parameters()):
    if 'conv_stem' in name or i < freeze_layers:
        param.requires_grad = False

# Unfreeze the deeper layers
for i, (name, param) in enumerate(model.named_parameters()):
    if 'conv_stem' not in name and i >= freeze_layers:
        param.requires_grad = True

# Check which parameters are trainable
for name, param in model.named_parameters():
    print(name, param.requires_grad)


features.0.0.weight False
features.0.1.weight False
features.0.1.bias False
features.1.0.block.0.0.weight False
features.1.0.block.0.1.weight False
features.1.0.block.0.1.bias False
features.1.0.block.1.fc1.weight False
features.1.0.block.1.fc1.bias False
features.1.0.block.1.fc2.weight False
features.1.0.block.1.fc2.bias False
features.1.0.block.2.0.weight False
features.1.0.block.2.1.weight False
features.1.0.block.2.1.bias False
features.1.1.block.0.0.weight False
features.1.1.block.0.1.weight False
features.1.1.block.0.1.bias False
features.1.1.block.1.fc1.weight False
features.1.1.block.1.fc1.bias False
features.1.1.block.1.fc2.weight False
features.1.1.block.1.fc2.bias False
features.1.1.block.2.0.weight False
features.1.1.block.2.1.weight False
features.1.1.block.2.1.bias False
features.2.0.block.0.0.weight False
features.2.0.block.0.1.weight False
features.2.0.block.0.1.bias False
features.2.0.block.1.0.weight False
features.2.0.block.1.1.weight False
features.2.0.block.1.1.bia

In [ ]:
model = models.efficientnet_b2( weights='IMAGENET1K_V1')
# Freeze model weights
for param in model.parameters():
    param.requires_grad = False

print(model.parameters())

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 142MB/s]


<generator object Module.parameters at 0x7f28c1f41690>


In [9]:
print(model.parameters())

<generator object Module.parameters at 0x7d95a7322ea0>


In [ ]:
print(model)


In [19]:
# Add on classifier
model.classifier[1] = nn.Linear(1408, 200)

total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

7,982,794 total parameters.
6,564,930 training parameters.


In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training the model
num_epochs = 20
training_loss_list = []
validation_accuracy_list = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        # print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloaders['train'])
    training_loss_list.append(average_loss)

    # Validation after each epoch
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for inputs, labels in dataloaders['test']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    validation_accuracy_list.append(accuracy)

    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss:.4f}, Test Accuracy: {accuracy:.4f}")


cuda


KeyboardInterrupt: 

In [ ]:


# Initialize model and move to device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training the model
num_epochs = 20
training_loss_list = []
validation_accuracy_list = []
training_accuracy_list = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    average_loss = total_loss / len(dataloaders['train'])
    training_loss_list.append(average_loss)
    training_accuracy = 100 * correct_train / total_train
    training_accuracy_list.append(training_accuracy)

    # Validation after each epoch
    model.eval()
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for inputs, labels in dataloaders['test']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    validation_accuracy_list.append(accuracy)

    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {average_loss:.4f}, Train Accuracy: {training_accuracy:.2f}%, Test Accuracy: {accuracy:.4f}")


Epoch 1/20, Average Training Loss: 2.9075, Train Accuracy: 32.40%, Test Accuracy: 0.5644
Epoch 2/20, Average Training Loss: 1.2344, Train Accuracy: 64.98%, Test Accuracy: 0.6831
Epoch 3/20, Average Training Loss: 0.7659, Train Accuracy: 77.71%, Test Accuracy: 0.6980
Epoch 4/20, Average Training Loss: 0.5413, Train Accuracy: 83.82%, Test Accuracy: 0.7302
